In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [7]:
df = pd.read_parquet("../data/combined_data.parquet")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29148075 entries, 0 to 29148074
Data columns (total 4 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   MeasureDate      object
 1   MeasurementName  object
 2   Value            object
 3   Status           object
dtypes: object(4)
memory usage: 889.5+ MB
None


In [12]:
df["MeasurementName"].value_counts()

MeasurementName
URB Lysimeter outside tension kPa              388641
URB Lysimeter inside tension kPa               388641
URB Lysimeter outside tension reference kPa    388641
URB Lysimeter vacuum kPa                       388641
URB Lysimeter level cm                         388641
                                                ...  
Weather Station radiation W/qm                 388641
Weather Station air temperature degC           388641
Weather Station temperature (5+) degC          388641
Weather Station wind direction deg             388641
Weather Station precipitation mm               388641
Name: count, Length: 75, dtype: int64

In [11]:
df.isnull().sum()


MeasureDate        75
MeasurementName     0
Value              75
Status              0
dtype: int64

In [13]:
# next step pivot?